In [74]:
## Arxiv 

from langchain_community.tools import ArxivQueryRun ,   WikipediaQueryRun 

from langchain_community.utilities import WikipediaAPIWrapper ,ArxivAPIWrapper

api = WikipediaAPIWrapper(top_k_results=1, doc_context_chars_mx =250)
wiki =  WikipediaQueryRun(api_wrapper =api)
wiki.name

'wikipedia'

In [75]:
api_arxiv = ArxivAPIWrapper(top_k_results=1, doc_context_chars_mx =250)
arxiv =  ArxivQueryRun(api_wrapper =api_arxiv)
arxiv.name

'arxiv'

In [93]:
tools = [wiki]#, arxiv]

In [94]:
import streamlit as st
import os
import time
import tempfile
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader ,PyPDFDirectoryLoader
from langchain_community.document_loaders import WebBaseLoader
load_dotenv()


True

In [95]:

os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HF_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "PDF ChatBot"

model = ChatGroq(
    model_name="llama-3.1-8b-instant",
    temperature=.75 ,max_tokens=250 ,
    api_key=os.getenv("GROQ_API_KEY") )



embeddings = HuggingFaceEndpointEmbeddings(
    task="feature-extraction",
    repo_id="sentence-transformers/all-MiniLM-L6-v2"
)


In [102]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Darshan_(Kannada_actor)")

docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).split_documents(docs)

vs = FAISS.from_documents(documents, embeddings)

retriver = vs.as_retriever()

retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEndpointEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022DF5930B90>, search_kwargs={})

In [103]:
from langchain.tools import tool

@tool
def langchain_search(query: str) -> str:
    """Search for information about LangChain"""
    return retriever.invoke(query)

print(langchain_search.name)


langchain_search


In [104]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("placeholder", "{chat_history}"), # Optional: for memory
    ("human", "{input}"),
    # This is the missing piece:
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])


In [105]:
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor

agent = create_tool_calling_agent(model, tools, prompt)

agent


RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [106]:
from langchain_classic.agents import AgentExecutor


age_executor = AgentExecutor(agent=agent, tools =tools, verbose=True)

age_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [109]:
age_executor.invoke({"input":"Who is Darshan (Kannada actor)"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Darshan (Kannada actor)'}`


Page: Darshan (Kannada actor)
Summary: Hemanth Kumar (born 16 February 1977), better known by his stage name Darshan Thoogudeepa or simply Darshan, is an Indian actor, film producer and distributor who works predominantly in Kannada cinema.
In a career spanning over two decades, Darshan has worked in over 50 films. He  is one of the highest paid actors and leading contemporary actors of Kannada films. Darshan has a significant fan following and is referred to by his fans as Challenging Star and D Boss.
Darshan established the production house Thoogudeepa Productions in 2006. Its first production was Jothe Jotheyali, with Darshan in a special appearance. His performances in Anatharu (2007) and Krantiveera Sangolli Rayanna (2012) won him praise from critics; his performance in the latter as the 19th-century warrior Sangolli Rayanna won him the Karnataka State Film Award for Best A

{'input': 'Who is Darshan (Kannada actor)',
 'output': "If you're looking for information about the actor Darshan, the above response should have provided you with sufficient information. If you'd like more information or clarification on a specific point, feel free to ask."}

In [110]:
reponse = age_executor.invoke({"input":"Who is Darshan (Kannada actor)"})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Darshan (Kannada actor)'}`


Page: Darshan (Kannada actor)
Summary: Hemanth Kumar (born 16 February 1977), better known by his stage name Darshan Thoogudeepa or simply Darshan, is an Indian actor, film producer and distributor who works predominantly in Kannada cinema.
In a career spanning over two decades, Darshan has worked in over 50 films. He  is one of the highest paid actors and leading contemporary actors of Kannada films. Darshan has a significant fan following and is referred to by his fans as Challenging Star and D Boss.
Darshan established the production house Thoogudeepa Productions in 2006. Its first production was Jothe Jotheyali, with Darshan in a special appearance. His performances in Anatharu (2007) and Krantiveera Sangolli Rayanna (2012) won him praise from critics; his performance in the latter as the 19th-century warrior Sangolli Rayanna won him the Karnataka State Film Award for Best A